In [12]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import tabulate
from IPython.display import HTML, display, Markdown
import csv
import unicodedata
import getopt
from ipywidgets import HBox, VBox, Layout
from matplotlib.ticker import MaxNLocator

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args


def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_4D.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    

### Table of Contents
[1. Transportation Sector Overview](#transportation)
<br>
&ensp;[1.1 Passenger Cars and Trucks](#light-duty-vehicles)
<br>
&ensp;[1.2 On-Road Freight and Heavy-Duty Passenger Vehicles](#heavy-duty-vehicles)
<br>
&emsp;[1.2.1 Trucks](#trucks)
<br>
&emsp;[1.2.2 Buses](#buses)
<br>
&ensp;[1.3 Off-Road Freight and Heavy-Duty Passenger Vehicles](#heavy-duty-vehicles)
<br>
&emsp;[1.3.1 Marine shipping](#marine)
<br>
&emsp;[1.3.2 Rail](#rail)
<br>
&emsp;[1.3.3 Aviation](#aviation)
<br>
&emsp;[1.3.3 Off-Highway](#off-highway)
<br>
&ensp;[1.4 Existing transportation capacity and future demand](#demand)
<br>
&ensp;[1.5 Charging and refueling infrastructure](#refueling)
<br>
&ensp;[1.6 Technology/commodity description lookup tool](#description_look_up)
<br>
&ensp;[1.7 Network diagram lookup tool](#network_look_up)
<br>
&ensp;[1.8 Technology/commodity look-up tool](#lookup_tool)
<br>
&ensp;[1.9 Demand Technology Specification](#demand_technologies)
<br>
&emsp;[1.9.1 Investment costs](#cost_invest)
<br>
&emsp;[1.9.2 Fixed costs](#cost_fixed)
<br>
&emsp;[1.9.3 Variable costs](#cost_variable)
<br>
&emsp;[1.9.4 Efficiency](#efficiency)
<br>
&emsp;[1.9.5 Existing Capacity](#exist_cap)
<br>
&emsp;[1.9.6 Discount Rate](#discount_rate)



### 1. Transportation Sector Overview <a class="anchor" id="transportation"></a>

The modeled transport modes are light-duty vehicles, trucks, buses, rail passenger, rail freight, subways, aviation, marine, and off-highway. The demands, efficiencies, and costs for these transport modes are largely drawn from the USEPA9r database [(Shay et al., 2013)](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883) out to 2050 and can be met by various technologies. To access the input workbooks from the USPA9r database, contact Carol Lenox (lenox.carol@epa.gov). We've made significant updates to this dataset, as described below.

The transportation sector in the OEO database can be split into three categories: light-duty, medium-duty, and heavy-duty vehicles.  Light, medium, and heavy-duty vehicle technology names in the database start with T_LDV, T_MDV, or T_HDV, respectively. Demand commodity names follow the format TMD(L,M,H)DV_. For example, small SUV (SS) demand is represented as TMDLDV_SS, where TMD stands for transportation miles demanded.

We represent seven light-duty vehicle size classes, outlined in [Section 1.1](#light-duty-vehicles). We externally constrain the percent of end-use demand met by each size class. As Temoa optimizes for the least-cost solution, without this constraint, the model would deploy only cheap, fuel efficient minicars and compact cars. We base size-class demand distributions on the MARKAL USEPA9r database [(Shay et al., 2013)](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883).

Medium-duty vehicles consist only of medium-duty trucks, corresponding to class 6 and 7 trucks.

Heavy-duty vehicles include heavy trucks, buses, passenger and freight rail, aviation, marine vessels, and off-highway vehicles. Each transportation technology meets a separate, exogenously specified end-use demand.  Depending on the technology, the demand may be in ton-miles, passenger-miles, or vehicle-miles traveled. Temoa does not currently allow for modal switching. For example, a long-haul truck cannot meet the same freight demand as a train; rather, the two have distinct exogenously defined demand profiles. Existing work shows that modal switching is one effective means of reducing transportation energy consumption, and we hope to incorporate modal models into the Temoa framework in the future [(Kaack et al, 2018)](https://iopscience.iop.org/article/10.1088/1748-9326/aad56c/meta).



We define technology-specific discount rates, or hurdle rates, for some technologies. Hurdle rates allow the modeler to account for consumer preferences otherwise ignored in a least-cost optimization model [2]. In the light-duty vehicle sector, we assume that conventional and hybrid gasoline-fueled vehicles (excluding plug-in hybrid vehicles) have a 5% hurdle rate, equal to the model's global discount rate, and all other powertrains have a 7.5% discount rate. For heavy-duty vehicles, conventional technologies have a 5% discount rate and alternative-fueled vehicles have a 10% discount rate.

#### 1.1 Passenger Cars and Trucks <a class="anchor" id="light-duty-vehicles"></a>

Light-duty vehicles have the most detailed representation. Seven different size classes are specified for light-duty vehicles and each size class is constrained to contribute a fraction of total light-duty demand. We optimize the technologies and fuels within each size class. The size classes are mini compact, compact, full, small SUV, large SUV, minivan, and pickup, and their share in meeting light duty demand is 2.3%, 28%, 36%, 11.5%, 10%, 3.8% and 8.4%. We assume these shares remain the same throughout time. Table 1 outlines the combinations of vehicle size classes and fuels that are represented in the current database.

**Table 1.** Light-duty vehicle size class and fuel combinations represented in the database.

|                                        | Minicar | Compact | Full | Small SUV | Large SUV | Van | Pickup |
|----------------------------------------|---------|---------|------|-----------|-----------|-----|--------|
| CNG                                    |         | X       | X    |           |           | X   | X      |
| Diesel Hybrid                          |         | X       | X    | X         | X         | X   |        |
| Diesel                                 |         | X       | X    | X         | X         | X   | X      |
| E10                                    | X       | X       | X    | X         | X         | X   | X      |
| E10 Hybrid                             |         | X       | X    | X         | X         | X   | X      |
| Plug-in Hybrid: 10-mile electric range |         | X       | X    | X         |           | X   |        |
| Plug-in Hybrid: 40-mile electric range |         | X       | X    | X         |           | X   | X      |
| E85X                                   |         | X       | X    | X         | X         | X   | X      |
| EV: 100-mile range                     | X       | X       | X    |           |           | X   |        |
| EV: 200-mile range                     | X       | X       | X    | X         | X         | X   |        |
| EV: 300-mile range                     | X       | X       | X    | X         | X         |     |        |
| H2 Fuel Cell                           |         | X       | X    | X         |           | X   |        |

#### 1.2 On-Road Freight and Heavy-Duty Passenger Vehicles <a class="anchor" id="heavy-duty-vehicles"></a>
#### 1.2.1 Trucks  <a class="anchor" id="trucks"></a>
Modeled truck technologies include heavy-duty short-haul, heavy-duty long-haul, medium-duty, and commercial.  In previous versions of the EPA database, heavy-duty trucking was split into short-haul and long-haul. This convention changed with v20.1.0; now the database only represents heavy-duty trucks in aggregate. The current OEO database retains a distinction between short- and long-haul heavy trucks, as the availability of low carbon technology differs between the two categories. As such, the techno-economic parameters are taken from an older version of the database (v18.1.4). This older version did not include electric trucks. The characterization of electric short-haul heavy-duty trucks (T_HDV_THSELC_N) is based on technoeconomic parameters from [NREL’s electrification futures study (EFS)](https://www.nrel.gov/docs/fy18osti/70485.pdf). The parameters match those for NREL’s “Heavy Duty BEV short-range” and are an average of the medium-, slow-, and rapid-advancement scenarios. Heavy-duty short-haul trucks are assumed to travel 27,390 miles per year, in accordance with EPA MARKAL’s assumptions for annual vehicle miles traveled (vmt). 

Heavy-duty long-haul truck cost and efficiency parameters are taken from the International Council on Clean Transportation and include characterization of hydrogen fuel cell and LNG long-haul trucks. We assume the average long-haul truck travels 79,520 miles per year, again based on EPA MARKAL. 

Similar to heavy-duty short-haul trucks, commercial truck efficiencies match v18.1.4 of the EPA databases save for electric trucks, which are drawn from NREL’s EFS. In this case, commercial trucks have the same technoeconomic parameters as NREL’s 300-mile range light-duty BEV truck [(NREL EFS)](https://www.nrel.gov/docs/fy18osti/70485.pdf). Medium-duty vehicles are consistent with USEPA9r. Table 2 lists the technologies currently represented for each truck type.

**Table 2.** Truck technologies and associated fuel / engine types.

| Truck Technology      | Fuel / Engine Types Represented                           
|-----------------------|----------------------------------------------------------
| Heavy Duty Short Haul | E10, E10 hybrid, diesel, diesel hybrid, B20, B20 hybrid, CNG, LPG, electricity        |
| Heavy Duty Long Haul  | diesel, diesel hybrid, B20, B20 hybrid, LPG, LNG, hydrogen                                         |
| Commercial            | E10, E10 hybrid, diesel, diesel hybrid, E85, hydrogen fuel cell, LPG, B20, B20 hybrid, electricity |
| Medium Duty           | E10, E10 hybrid, diesel, diesel hybrid, B20, B20 hybrid, CNG, LPG, electricity                     |

#### 1.2.2 Buses <a class="anchor" id="buses"></a>
We also model buses. In accordance with the latest EPAUS9r database, we distinguish between transport buses and school buses. Demands for each bus type are specified separately. Table 3 lists the types of buses currently represented in the OEO database.

**Table 3.** Bus technologies and associated fuel / engine types.

| Bus Type | Fuels / Engine Types Represented |
|----------|----------------------------------|
| Transit  | Diesel, diesel hybrid, E10, electricity, B20, CNG, hydrogen |
| School   | Diesel, diesel hybrid, E10, electricity, B20, LPG, CNG      |

#### 1.3 Off-Road Freight and Passenger Vehicles <a class="anchor" id="buses"></a>
#### 1.3.1 Marine shipping <a class="anchor" id="marine"></a>
When modeling marine options, it is important to distinguish important characteristics of particular vessel types, such as annual ton-miles (or passenger-miles) traveled and engine power, and also to ensure that decarbonization options are adequately represented. For example, [Minnehan and Pratt (2017)](https://energy.sandia.gov/wp-content/uploads/2017/12/SAND2017-12665.pdf) note that passenger ferries may be able to operate on a battery, gaseous hydrogen, liquid hydrogen, and methanol, while a large containership’s decarbonization options are more limited. The heavy-duty shipping sector includes passenger vessels as well as five freight vessel types: container ships, liquid bulk ships, general cargo ships, tugboats, and offshore supply vessels. These classifications are drawn from the annual publication of Waterborne Transportation Lines of the United States by the Institute for Water Resources (IWR), which provides data on the number of vessels, horsepower, and cargo or passenger capacity of U.S. flagged vessels [(U.S. Army Corps of Engineers Institute for Water Resources)](https://usace.contentdm.oclc.org/digital/collection/p16021coll2/id/3796/). The current OEO database uses only U.S. flagged vessels, and demand is only for domestic cargo and passenger transport. 

Most of the technical parameters for each vessel type were taken from the [European Environment Agency’s (EEA’s) Guidebook](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019) for International Maritime and Inland Navigation, National Navigation, National Fishing, and Recreational Boats. The EEA provides emission factors (EFs), average engine power ratings, average fuel consumption, and average load factor for multiple ship types. While EEA gives technical parameters for passenger vessels, their characterization includes large international cruise ships, which skews the estimate significantly. The IWR provides data on the main engine power rating for vessels in the domestic fleet; this data is used to determine the main engine power for domestic passenger vessels. The EEA estimates are used for freight vessels, as the process of pulling vessel engine characteristics from the IWR document for thousands of domestic vessels is time-consuming. Excel spreadsheets exist, but to our knowledge, the workbooks do not include vessel classification (e.g., offshore supply vessel, cargo), meaning individual vessels would have to be matched with their classification from the PDF report. This effort will be considered for future work. The IWR does not provide an estimate of auxiliary engine power, but the EEA estimates the ratio of auxiliary engine power to main engine power by ship type. This estimate is used to determine an approximate auxiliary engine power for passenger vessels.

For freight shipping, the total annual ton-miles traveled per vessel per year are estimated using an EPA workbook that pulls data from Lloyd’s register Fairplay as well as several literature studies [R. M. Weyl, R. Billings, and C. Gage, “MARKAL-MV ERG Final.” EPA Repository]. The workbook provides estimates of ton-miles traveled for containerships, general cargo ships, tankers (assumed to correspond to liquid bulk ships), tug boats, and “miscellaneous,” which is used to estimate freight work for offshore supply vessels. Annual passenger-miles traveled per vessel per year are estimated using the [American Public Transportation Associations’ Public Transportation Fact Book](https://www.apta.com/wp-content/uploads/APTA_Fact-Book-2019_FINAL.pdf), which reports passenger miles traveled (pmt) and vehicle miles traveled (vmt) by all ferries annually.

Efficiency for freight water transport is found using the equation below:

\begin{equation*}
Eff =  \frac{D}{sfc \cdot e \cdot EC}
\end{equation*}

Where 
- $D$: freight work (billion ton- or passenger- miles traveled annually)
- $e$: gravimetric energy density of the fuel (kWh/kg fuel)
- $sfc$: specific fuel consumption (kg fuel/ kWh)
- $EC$: annual energy consumption (PJ)

The gravimetric energy densities for MGO, RFO, and liquid hydrogen are given in Table 3.


**Table 3.** Conventional marine fuels and their energy densities.

|     Fuel                 |     Energy Density   (kWh/kg)    |     Source  |
|--------------------------|----------------------------------|-------------|
|     Marine gasoil       |     11.84                        |     [Raucci, 2017](https://discovery.ucl.ac.uk/id/eprint/1539941/)    |
|     Residual fuel oil    |     11.25                        |     [Raucci, 2017](https://discovery.ucl.ac.uk/id/eprint/1539941/)    |
|     Liquid hydrogen      |     33.33                        |     [Raucci, 2017](https://discovery.ucl.ac.uk/id/eprint/1539941/)    |


The specific fuel consumption values for marine gasoil- and residual fuel oil-fueled vessels come from the European Environment Agency’s guidebook. The guidebook reports sfc values by engine type (high-speed diesel, medium-speed diesel, slow-speed diesel, and gas turbine). Additionally, the guidebook reports the percent of each vessel type equipped with each engine type. This data is given in Table 4. Specific fuel consumption by engine type is reported in Table 5.

**Table 4.** Vessel engine types taken from the [European Environment Agency](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019).

|                                |     Marine Gasoil    |              |              |              |     Residual Fuel Oil    |              |              |              |
|--------------------------------|----------------------|--------------|--------------|--------------|--------------------------|--------------|--------------|--------------|
|     Ship category              |     SSD              |     MSD      |     HSD      |     GT       |     SSD                  |     MSD      |     HSD      |     GT       |
|     Liquid bulk ship           |     0.191            |     0.695    |     0.114    |     0.000    |     0.776                |     0.214    |     0.008    |     0.001    |
|     Container                  |     0.898            |     0.080    |     0.022    |     0.000    |     0.943                |     0.056    |     0.001    |     0.000    |
|     General Cargo              |     0.027            |     0.645    |     0.327    |     0.000    |     0.513                |     0.480    |     0.005    |     0.001    |
|     Offshore Supply Vessel*    |     0.010            |     0.628    |     0.354    |     0.008    |     0.569                |     0.371    |     0.056    |     0.004    |
|     Tug                        |     0.000            |     0.430    |     0.567    |     0.003    |     0.000                |     0.887    |     0.113    |     0.000    |
*Offshore supply vessels are not explicitly represented; qualities for vessel category “other” are used to represent these vessels*

**Table 5.** Specific fuel consumption (SFC) by engine type taken from the [European Environment Agency](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019).

|     Engine   type    |     Fuel   Type    |     SFC   [g/kWh]    |
|----------------------|--------------------|----------------------|
|     HSD              |     RFO            |     213              |
|                      |     MGO            |     203              |
|     MSD              |     RFO            |     213              |
|                      |     MGO            |     203              |
|     SSD              |     RFO            |     195              |
|                      |     MGO            |     185              |
|     GT               |     RFO            |     305              |
|                      |     MGO            |     290              |

Specific fuel consumption for each vessel type is found by multiplying the sfc for each engine type by the percent of vessels with that engine type and summing over all combinations of engine and fuel. The annual energy consumption is found by multiplying engine power by the load factor and the number of operating hours. We account for the two primary operating modes of ocean-going ships: at berth and cruising. Hours of operation for liquid bulk ships, container ships, and general cargo ships are estimated from an [International Council on Clean Transportation](https://theicct.org/publications/black-carbon-emissions-global-shipping-2015) report. Operational hours represent the global fleet; these values may differ slightly for the US domestic fleet. For offshore supply vessels and tugboats, we use work by [Lindstad et al. (2017)](https://www.sciencedirect.com/science/article/pii/S1361920916302723?via%3Dihub) and [Jayaram et al. (2010)](https://ww2.arb.ca.gov/sites/default/files/2020-12/hybridreport1010_remediated.pdf), respectively, to estimate time spent in each operating phase.   Table 5 summarizes the time spent in each operating phase per year for each vessel type. Average engine power for each vessel type are taken from EEA and reported in Table 7.

**Table 6.** Hours spent in each operational phase by vessel type

|     Vessel Type               |     Hours spent cruising/at berth    |     Source    |
|-------------------------------|--------------------------------------|---------------|
|     Liquid Bulk               |     3313 / 4963                      |     [Comer et al., 2015](https://theicct.org/publications/black-carbon-emissions-global-shipping-2015)      |
|     Container                 |     4940 / 3578                      |     [Comer et al., 2015](https://theicct.org/publications/black-carbon-emissions-global-shipping-2015)      |
|     General Cargo             |     2985 / 5083                      |     [Comer et al., 2015](https://theicct.org/publications/black-carbon-emissions-global-shipping-2015)      |
|     Offshore Supply Vessel    |     3490 / 5720                      |     [Lindstad et al., 2017](https://www.sciencedirect.com/science/article/pii/S1361920916302723?via%3Dihub)      |
|     Tug                       |     2239 / 5431                      |     [Jayaram et al., 2010](https://ww2.arb.ca.gov/sites/default/files/2020-12/hybridreport1010_remediated.pdf)      |


**Table 7.** Engine power ratings, drawn from the European Environment Agency

|     Vessel Type                |     Engine Power (kW)    |
|--------------------------------|--------------------------|
|     Liquid Bulk                |     6695                 |
|     Container                  |     22929                |
|     General Cargo              |     2657                 |
|     Tug                        |     2059                 |
|     Offshore Supply Vessel*    |     2778                 |

*Offshore supply vessels are not explicitly represented; qualities for vessel category “other” in the EEA guidelines are used to represent these vessels*

The load factor (LF) varies depending on operational phase. According to the [EEA guidebook](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019), when cruising, the LF is 0.8 and at berth it is 0.2 (except for liquid bulk ships, for which it is 0.1). We also assume that the efficiency of new ships increases in the future. The [Third IMO Greenhouse Gas Study](https://www.imo.org/en/OurWork/Environment/Pages/Greenhouse-Gas-Studies-2014.aspx) defines two efficiency improvement scenarios. The first assumes a 60% efficiency improvement in 2050 relative to 2012 levels and the second assumes a 40% efficiency improvement. We conservatively assume the latter, implementing it as a 5% efficiency improvement in every 5-year time period starting in 2020.

We base hydrogen vessel feasibility estimates on [Minnehan and Pratt (2017)](https://energy.sandia.gov/wp-content/uploads/2017/12/SAND2017-12665.pdf), which find that the majority of vessel classes are capable of running on liquefied hydrogen. Of the thirteen existing vessels considered in their analysis, they found ten would be able to run on liquid hydrogen based on voyage distance and mass and volume limitations. These vessels included a container ship, a general cargo ship, an offshore supply vessel, and a bulk cargo ship. Therefore, we define all freight shipping technologies with a liquid hydrogen option except tugboats, which were not considered in the study. Furthermore, a [2020 ICCT report](https://theicct.org/publications/zero-emission-container-corridor-hydrogen-2020) found that 99% of voyages made along the international shipping corridor between the San Pedro Bay and the Pearl River Delta could be fueled by hydrogen. We assume hydrogen ships will be available by 2030, based on an estimate from Flexport, according to an [NPR report](https://www.npr.org/2019/07/16/716693006/the-dawn-of-low-carbon-shipping#:~:text=via%20Getty%20Images-,Container%20ships%20and%20other%20maritime%20vessels%20currently%20run%20on%20pollutant,new%20fuels%20such%20as%20hydrogen). This is likely a conservative estimate, as some European countries plan to deploy hydrogen ships as early as 2021 ([see this article](https://www.offshore-energy.biz/flagships-project-gets-usd-5-6-mn-for-two-hydrogen-fueled-ships/#:~:text=Flagships%20Project%20Gets%20USD%205.6%20Mn%20for%20Two%20Hydrogen%2DFueled%20Ships,-Business%20%26%20Finance&text=The%20European%20innovation%20project%20Flagships,vessels%20in%20France%20and%20Norway.)).  To calculate vessel efficiencies, we use a slightly simplified version of the method outlined above. We use a single value for specific fuel consumption, taken from [Raucci  (2017)](https://discovery.ucl.ac.uk/id/eprint/1539941/), rather than a weighted average. Otherwise, the calculation methodology is the same.

Efficiency calculations for passenger vessels deviate from the EEA methodology as the EEA report includes large ocean-going cruise ships in their passenger vessel classification. The IWR reports only one such vessel in the US merchant fleet, so EEA values are not appropriate. Passenger vessel efficiency is based on [Cottrell (2011)](https://www.mdpi.com/1996-1073/4/2/239), whose work compared US ferry boat efficiencies to highway bridges crossing the same body of water. Cottrell’s work gives passenger-miles per gallon for eight ferry vessel companies operating 96 marine gasoil-powered ferries. Using this data yields an average efficiency of 0.092 billion passenger miles traveled per PJ. To benchmark both passenger and freight vessel efficiencies, as well as demand estimates, we compare total energy consumption for domestic shipping from our calculations to Annual Energy Outlook forecasts.

We don’t consider hydrogen-powered passenger vessels, but model electric passenger vessels based on the E-Ferry, an electric ferry currently operating in Southern Denmark. Based on energy consumption data from an EU-sponsored evaluation report on the E-Ferry, efficiency is found to be 0.165 billion passenger miles per PJ.


Emission factors (kt/ billion ton- or passenger-miles traveled) are calculated using the EEA Tier II methodology. Similarly to efficiency, emission factors are reported by engine type.  The emission factors for each vessel type are calculated using the percent of each vessel type using a particular engine and are reported in Table 8.

**Table 8.** Emission Factors, kg/tonne fuel [European Environment Agency](https://www.eea.europa.eu/publications/emep-eea-guidebook-2019).

|                         |     Fuel     |     NOx EF, 2000     |     NOx EF, 2005     |     NOx EF, 2010     |     PM10     |     PM2.5     |     NMVOC     |
|-------------------------|--------------|----------------------|----------------------|----------------------|--------------|---------------|---------------|
|     Liquid Bulk         |     RFO      |     86.62            |     83.84            |     80.75            |     7.60     |     6.81      |     2.7       |
|                         |     MGO      |     69.46            |     67.28            |     64.91            |     1.52     |     1.34      |     2.8       |
|     General Cargo       |     RFO      |     79.53            |     76.91            |     74.16            |     6.31     |     5.66      |     2.7       |
|                         |     MGO      |     63.81            |     61.84            |     59.51            |     1.50     |     1.31      |     2.8       |
|     Container           |     RFO      |     91.24            |     88.35            |     85.05            |     8.42     |     7.55      |     2.7       |
|                         |     MGO      |     89.02            |     85.86            |     83.73            |     1.59     |     1.48      |     2.8       |
|     Tug                 |     RFO      |     65.01            |     62.76            |     60.66            |     3.80     |     3.40      |     2.7       |
|                         |     MGO      |     61.52            |     59.56            |     57.35            |     1.50     |     1.30      |     2.8       |
|     Offshore Supply     |     RFO      |     80.62            |     77.99            |     75.17            |     6.58     |     5.89      |     2.7       |
|                         |     MGO      |     62.82            |     60.88            |     58.57            |     1.49     |     1.29      |     2.8       |
|     Passenger           |     RFO      |     65.01            |     62.75            |     60.65            |     3.88     |     3.48      |     2.7       |
|                         |     MGO      |     48.13            |     46.61            |     44.85            |     0.99     |     0.86      |     2.8       |

The values in Table 8 are converted from kg/tonne fuel to kt/btm or kt/bpm by multiplying the specific fuel consumption (g/kWh), the total annual energy consumption (kWh/year) and the ton- or passenger-miles traveled in a year. SO2 emissions are calculated using the following equation:

\begin{equation*}
 EF_{SO_{2}} = 0.02 \cdot \%_{sulfur} \cdot 0.98 \cdot sfc
\end{equation*}

This equation is taken from the [Third IMO Greenhouse Gas Study](https://www.imo.org/en/OurWork/Environment/Pages/Greenhouse-Gas-Studies-2014.aspx), which includes the constant 0.98 as approximately 98% of fuel sulfur is converted to gaseous SO2. As of January 1, 2015, ships inside a sulfur emissions control area must use fuel or an equivalently strict emissions control technology that limits emissions to less than or equal to 0.1% m/m (that is, mass of pollutant by mass of fuel) [(Sulphur oxides (SOx) – Regulation 14)](https://www.imo.org/en/OurWork/Environment/Pages/Sulphur-oxides-(SOx)-%E2%80%93-Regulation-14.aspx). To meet sulfur requirements, ships can use residual fuel oil (RFO) with a scrubber, marine gasoil (MGO), or an alternative fuel such as liquefied natural gas (LNG) or hydrogen [(Zis et al., 2016)](https://journals.sagepub.com/doi/abs/10.3141/2549-05?journalCode=trra).  Additionally, ships must meet nitrogen oxide standards based on their construction date (see Table 9). It is assumed here that all existing vintages meet Tier II standards while new builds meet Tier III standards and that all vessels meet the sulfur standard, since it was implemented before the model’s first time period (2017). To meet NOx requirements, selective catalytic reduction (SCR) or exhaust gas recirculation (EGR) technologies can be retrofitted onto vessels or alternative fuels may be used [(Jonge et al., 2005a)](https://ec.europa.eu/environment/air/pdf/task2_nox.pdf). For simplicity, we assume that all vessels using MGO are equipped with SCR technology and all vessels using RFO are equipped both with a scrubber and SCR. Relevant assumptions for emission factor calculations are that scrubbers reduce SO2 by 75% and PM by 25% from the ship and engine type baseline and selective catalytic reduction (SCR) reduces NOx by 90% [(Jonge et al., 2005a](https://ec.europa.eu/environment/air/pdf/task2_nox.pdf); [Jonge et al., 2005b)](https://ec.europa.eu/environment/air/pdf/task2_so2.pdf).

**Table 9.** International Maritime Organizstion (IMO) NOx Emissions Standards.

|     Tier    |     Ship   construction date on or after    |     <td colspan=3>**Total Weighted cycle emission limit   (g/kWh) n = engine’s rated speed (rpm)**    |                       |                  |
|-------------|---------------------------------------------|-------------------------------------------------------------------------------------|-----------------------|------------------|
|             |                                             |     n < 130                                                                         |     n = 130 – 1999    |     n >= 2000    |
|     I       |     1 January 2000                          |     17.0                                                                            |     45*n^-0.2          |     9.8          |
|     II      |     1 January 2011                          |     14.4                                                                            |     44*n^-0.23         |     7.7          |
|     III     |     1 January 2016                          |     3.4                                                                             |     9*n^-0.2           |     2.0          |



Freight vessel costs are derived from [Taljegard et al. (2014)](https://pubs.acs.org/doi/abs/10.1021/es5018575).  Costs are given for deep sea, short sea, and container ships. We assume that liquid bulk ships and general cargo ships are deep sea ships while offshore vessels and tugboats are short sea ships. Costs are reported by Taljegard et al. (2014) in USD/kW. We convert the cost to \\$M/(billion ton-mile/year) or \\$M/(billion passenger-mile/year) by multiplying by the average engine size (kW) for each vessel type as reported by the [ICCT](https://theicct.org/publications/black-carbon-emissions-global-shipping-2015) and dividing by the annual ton-miles traveled by each vessel type.

Operation and maintenance costs for RFO / MGO-fueled vessels are assumed to be 5% of capital costs in accordance with [EPA MARKAL](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883). We also include costs for emission control technologies (both investment and operation \& maintenance), which are taken from Entec UK Limited [(Jonge et al., 2005a](https://ec.europa.eu/environment/air/pdf/task2_nox.pdf); [Jonge et al., 2005b)](https://ec.europa.eu/environment/air/pdf/task2_so2.pdf). It is assumed that selective catalytic reduction technology costs $0.5M and a scrubber costs \\$6.5M. Operation and maintenance costs depend on the fuel being used. For RFO-fueled vessels, SCR O\&M costs are \\$7.3/MWh. For MGO-fueled vessels, they are \\$4.2/MWh. Scrubber O\&M costs are set at \\$0.778/MWh. Costs are converted from 2004 Euros [assuming 1.24 USD to Euro 2004](https://www.macrotrends.net/2548/euro-dollar-exchange-rate-historical-chart) and [1.255 USD 2004 to USD 2015](https://www.minneapolisfed.org:443/about-us/monetary-policy/inflation-calculator).


Passenger vessel costs are based on the [E-Ferry report](https://www.xn--el-frgeprojekt-3ib.dk/nyheder/e-ferry-ellen-crosses-the-finish-line-and-delivers-great-results), which compares the novel electric ferry to existing marine gasoil ferries. A passenger vessel is assumed to cost 12,855,657 2019 EUR, which is converted to \\$16546M USD 2015/bpm. Operation and maintenance costs are reported to be \\$2455M USD 2015/bpm (converted from 1,907,618 EUR 2019). Emission control technologies are also added to passenger vessel costs.

#### 1.3.2 Rail  <a class="anchor" id="rail"></a>
We model two general categories of rail technologies: passenger and freight. In accordance with EPA MARKAL, passenger rail is split into intercity rail, subways, commuter rail, which meet a final end demand "TMDHDV_RP". In the US, intercity rail represents all Amtrak lines, while commuter rail refers to shorter distance public transportation railways. Subway, intercity, and freight rail efficiencies are taken from [GREET](https://greet.es.anl.gov/) and commuter rail efficiencies are estimated based on simulations included in [Isaac et al. (2020)](https://ncst.ucdavis.edu/research-product/fuels-and-fuel-technologies-powering-21st-century-passenger-and-freight-rail).

Passengers per vehicle mile traveled (i.e., load factor) is calculated as the ratio of total passenger miles traveled to total vehicle miles traveled in the United States. This value is found using data from the [Public Transportation Fact Book](https://www.apta.com/research-technical-resources/transit-statistics/public-transportation-fact-book/).  Efficiency for dual-mode trains is assumed to be the average of diesel and electric efficiencies as a first pass estimate until better data is found. Table 10 outlines the fuels included for each category of rail technology.

**Table 10.** Fuels available for rail by category.

|     Rail Category             |     Fuels represented                                                         |
|-------------------------------|-------------------------------------------------------------------------------|
|     Freight Rail              |     Diesel, B20, B100,   LNG                                                  |
|     Commuter Rail             |     Diesel, electric,   B20, dual-mode (can operate as diesel or electric)    |
|     Intercity rail            |     Diesel, B20, B100,   hydrogen                                             |
|     Subways and streetcars    |     Electricity                                                               |

Capital costs are estimated using several sources. Freight rail and intercity rail capital costs are taken from [Isaac and Fulton (2016)](https://steps.ucdavis.edu/wp-content/uploads/2017/10/RAPHAELISAAC-2017-UCD-ITS-RP-17-02-2.pdf). Variable costs are set to 6% of capital costs for each technology type based on [EPA MARKAL](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883). For commuter rail, [Gattuso and Restuccia (2014)](https://www.sciencedirect.com/science/article/pii/S1877042814000895) present estimates for several different commuter rail trains, each with differ passenger capacity and number of train cars. We assume a representative train has three cars and as such, costs are adjusted to a three-car baseline. For example, Gattuso and Restuccia (2014) report that the AnsaldoBreda RegioStar has 8 cars and costs \\$8.55M USD 2015 (converted from M EUR 2013). This is converted to a three-car basis by multiplying the total train cost by 3/8. Additional data is collected from several news articles noting the purchase of electric and diesel multiple unit trains [(Metrarail, ](https://metrarail.com/about-metra/newsroom/facts-about-metras-highliner-purchase ) [Denver Post)](https://www.denverpost.com/2014/12/02/rtds-new-rail-cars-for-fastracks-system-unveiled-to-public/). Currently, the database does not contain operation and maintenance costs for commuter rail.

#### 1.3.3 Aviation  <a class="anchor" id="aviation"></a>
Per the USEPA9r MARKAL database, aviation is split into passenger jets and general aviation, both of which are passenger modes. We currently do not model freight aviation,  General aviation is fueled by gasoline and passenger jets by jet fuel. Other than synthetic fuels, there are currently no alternative fuels for aviation characterized in the database. All efficiency and cost parameters are drawn from the USEPA9r database. 

#### 1.4 Existing transportation capacity and future demand <a class="anchor" id="demand"></a>
The <a href="https://www.nrel.gov/docs/fy18osti/71500.pdf" rel="nofollow">NREL Electrification Futures Study (EFS)</a> report presents a range of demand scenarios for all the energy sectors in the United States. Specifically, the study estimates transportation-related service demands, as shown in Table 11, primarily by downscaling service demands data from the US EIA 2017 Annual Energy Outlook and the NEMS model. We draw transportation service demands primarily from EFS. Because the model optimizes the fuels used to meet end-use transportation demands, we aggregate the fuel-specific EFS values into a single annual demand in each region for each demand category. Exceptions are discussed below.

Unless stated otherwise, the same EFS data is used to estimate existing capital stock in the transportation sector. We use EFS’s 2017 demand values by fuel and state (which we aggregate to the OEO region) to estimate existing capacity (billion vmt/yr). For simplicity, existing capacity is divided evenly between all existing vintages of a particular technology. 
 
 **Table 11.** Annual service demand categories in the OEO database, demand units, and the associated demand description. 

| Demand category | Unit | OEO demands |
|-|-|-|
| Aviation <img width=200/>| billion seat-miles | transport passenger miles demanded for HDV airplane|
| Freight rail | billion ton-miles | transport miles demanded for HDV rail freight |
| Heavy duty trucks | billion vehicle-miles | transport miles demanded for HDV heavy truck long haul, transport miles demanded for HDV heavy truck short haul |
| Light duty autos | billion vehicle-miles | transport miles demanded for LDV|
| Light duty trucks | billion vehicle-miles | transport miles demanded for HDV commercial truck |
| Medium duty trucks | billion vehicle-miles| transport miles demanded for medium duty truck |
| Passenger rail | billion passenger-miles | transport miles demanded for HDV rail passenger |
| School and intercity buses | billion passenger-miles | transport miles demanded for school bus |
| Transit buses | billion vehicle-miles | transport miles demanded for transit bus |
| Off-highway vehicles | billion vehicle-miles | transport miles demanded for off-highway vehicles (such as agricultural equipment)|
| Freight ships | billion ton-miles | transport miles demanded for freight ships |
| Passenger ships | billion passenger-miles | transport miles demanded for passenger ships | 

#### Light-duty vehicles
While total demand and existing capacity are drawn from EFS, it does not report existing capacity by vehicle class. Temoa distinguishes between seven vehicle size classes (mini-compact, compact, small SUV, large SUV, full, minivan, and pickup). The distribution of demand and existing capacity by vehicle type is drawn from <a href="https://cfpub.epa.gov/si/si_public_record_report.cfm?Lab=NRMRL&dirEntryId=278925" rel="nofollow">EPA MARKAL</a> and is assumed to be the same region-to-region and stay constant across the model’s time horizon. 

#### Short- and long-haul heavy-duty trucks
EFS does not distinguish between short- and long-haul heavy trucks. We again use total demand and existing capacity from EFS, but rely on <a href="https://cfpub.epa.gov/si/si_public_record_report.cfm?Lab=NRMRL&dirEntryId=278925" rel="nofollow">EPA MARKAL </a>to divide demand between short- and long-haul vehicles. In line with MARKAL, we assume that 41% of vmt demand is met by short-haul vehicles and that 59% is met by long-haul vehicles in each time period. Following EPA MARKAL’s convention, we assume that all existing long-haul trucks are fueled by diesel. Therefore, all other heavy-truck capacity in EFS (CNG, E10, LPG) is assumed to be short-haul. 

#### School and Transit Buses
EFS does not report 2017 school bus passenger miles traveled by fuel type. We use the total paseenger mile demand from EFS but estimate existing capacity by fuel type using the <a href="https://www.eia.gov/outlooks/aeo/pdf/AEO2020%20Full%20Report.pdf" rel="nofollow">2020 Annual Energy Outlook</a>. Transit bus existing capacity and demand are consistent with EFS.

#### Freight rail
Rather than using EFS for rail freight, we use data from the Freight Analysis Framework <a href="https://ops.fhwa.dot.gov/freight/index.cfm" rel="nofollow">(FAF)</a>. Similar to EFS, this dataset projects annual demand in ton-miles traveled by state. FAF does not report a 2017 demand value, so we assume that existing capacity is equivalent to 2012 demand. The FAF does not project out to 2050; projections stop at 2045. We calculate the average five-year growth rate from 2020 to 2045 and apply it to 2045 to estimate 2050 demand.

#### Aviation
EFS models only one demand for aviation, but we follow MARKAL’s convention and divide aviation into “general aviation” and “passenger jets.” Following EPA MARKAL, we assume that in all regions, 99% of capacity and demand is passenger jets, with the remaining 1% allocated to general aviation. As mentioned above, we do not currently model freight aviation. It is estimated that globally, aviation accounts for only [1% of total freight tonnage](https://www.eesi.org/papers/view/fact-sheet-the-growth-in-greenhouse-gas-emissions-from-commercial-aviation#6). We hope to include freight aviation in a future iteration of the database.

#### Freight ships
EFS does not report freight shipping. As a result, we use the <a href="https://ops.fhwa.dot.gov/freight/index.cfm" rel="nofollow">Freight Analysis Framework</a> for existing capacity and demand breakdowns. FAF does not distinguish between different vessel types. We use data from the United States Army Corps of Engineers report, [‘Waterborne Transportation Lines of the United States’](https://usace.contentdm.oclc.org/digital/collection/p16021coll2/id/1376/) to estimate the breakdown of freight cargo by vessel type. We assume these fractions remain constant out to 2050.

#### Passenger ferries
Neither MARKAL nor EFS report data on passenger ferries. We estimate existing capacity and future demand based on the <a href="https://www.apta.com/research-technical-resources/transit-statistics/public-transportation-fact-book/" rel="nofollow">American Public Transportation Fact Book (APTFB)</a>. We do not have forecasts for passenger ferry miles traveled, so assume a flat demand out to 2050. According to the APTFB, in 2019, passenger ferries consumed approximately 46 million gallons of diesel fuel. Assuming an energy content of <a href="https://www.engineeringtoolbox.com/energy-content-d_868.html" rel="nofollow">139,000 BTU/gallon</a>, ferries in the US only consumed less than 1 quad in 2019. We assume the regional breakdown of demand is identical to that for freight ships. 

#### Passenger rail
EFS only reports “passenger rail,” but we model three subcategories: commuter rail, intercity rail, and subways/streetcars. <a href="https://cfpub.epa.gov/si/si_public_record_report.cfm?Lab=NRMRL&dirEntryId=278925" rel="nofollow">EPA MARKAL</a>, on the other hand, has demand and capacity disaggregated by the three subcategories. The fraction of total passenger rail demand met by each of the three subcategories varies region-to-region, as some regions are more dependent on one mode than the others. We use MARKAL data for existing capacity and demand, rather than EFS, as it is the most disaggregated. While existing capacity is largely drawn from EPA MARKAL, the MARKAL database does not model dual-mode commuter rail locomotives. These locomotives can run on diesel, but switch over to electric power where the infrastructure is available or where local air quality ordinances require it. To estimate the fraction of existing commuter rail that is dual mode, rather than diesel or electric, we use data on annual ridership by region from the <a href="https://www.apta.com/wp-content/uploads/2019-Q4-Ridership-APTA.pdfAmerican" rel="nofollow">Public Transportation Association</a>. We combine this with estimates of the number of locomotives powered by diesel, electricity, or both from each of the 32 commuter rail lines across the US. This data is largely drawn from the websites of individual transit organizations. While the number of locomotives is not a perfect proxy for the number of passenger miles traveled by each type, we assume it is sufficient as all new capacity will be optimized by the model and this assumption only affects the existing stock.
    
#### Off-highway vehicles
Currently, off-highway vehicle demand is drawn from EPA MARKAL. We do not represent individual vehicle types, but follow EPA's convention of meeting off-highway demand with two off-highway technologies, off-highway diesel and off-highway gasoline. 

End-use regional OEO demand projections can be explored in the tool below.

In [13]:
def show_demand(con):

    display_types = ['table', 'figure']

    df_demand = pd.read_sql("SELECT * FROM Demand WHERE demand_comm in (SELECT comm_name FROM commodities WHERE flag='d') \
                            AND demand_comm in (SELECT output_comm FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector='transport'))", con)
    
    df_tech_desc = pd.read_sql("SELECT comm_name, comm_desc FROM commodities", con)
    df_demand['demand_comm'] = df_demand['demand_comm'].map(lambda x: df_tech_desc.loc[df_tech_desc.comm_name==x,'comm_desc'].values[0].replace('#','').strip())
    
    demand_comms = []
    for unique_demand_comm in df_demand.demand_comm.unique():
        demand_comms.append(unique_demand_comm)
    demand_comms = list(set(demand_comms))
    regions = df_demand.regions.unique()


    def filter_demand_comm(demand_comm='', o_format=''):
        demand_comm_name = pd.read_sql("SELECT comm_desc FROM commodities where comm_name=='" + demand_comm +"'", con)
        df_sel = df_demand[(df_demand.demand_comm.str.contains(demand_comm))]
        df_sel = df_sel.pivot_table(
            index=['regions', 'demand_comm', 'demand_units'],
            columns='periods',
            values='demand').reset_index().set_index('regions')
        #df_sel['demand_comm'] = demand_comm_name.iloc[0,0].replace('#','').strip()
        df_sel.rename(columns={'demand_units': 'units', 'demand_comm':'demand'}, inplace=True)
        df_sel.units = df_sel.units.str.replace('#','').str.strip()
        if o_format == 'table':
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif o_format == 'figure':
            fig, ax = plt.subplots(figsize=(10, 8))
            df_sel.drop(columns=['demand', 'units']).transpose().plot.bar(stacked=True, ax = ax)
            plt.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
            handles, labels = ax.get_legend_handles_labels()
            ax.legend(handles[::-1], labels[::-1], loc='upper right',bbox_to_anchor=(1.2, 1))
            plt.ylabel(df_sel['demand'].unique()[0] + ' (' + df_sel.units.unique()[0].replace('#','') + ')')
            plt.xlabel('')

    display(Markdown('Annual end-use regional demand projections to 2050'))
    w2 = widgets.Select(options=demand_comms, description='demand')
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_demand_comm, demand_comm=w2, o_format=w3)


    controls_rows(w)

show_demand(con)

Annual end-use regional demand projections to 2050

#### 1.5 Charging and refueling infrastructure
##### Electric Vehicle Charging Stations

We model electric vehicle supply equipment (EVSE) for light- and heavy-duty vehicles. Light-duty EVSE costs are calculated as a weighted average of Level 2 (L2) at-home chargers, L2 public chargers, and DC fast chargers (DCFC). Heavy-duty EVSE costs are a weighted average of 50 kW, 150 kW, and 350 kW DCFC chargers. All light-duty capital cost assumptions are drawn from Borlaug et al (2020). To calculate a weighted average capital cost, we draw assumptions on the number of vehicles served by each charger type from a 2021 NREL report. The report finds that as of 2017, each public L2 charger could serve 20 vehicles while each DCFC serves 100 vehicles. We assume that at-home L2 chargers currently only serve a single vehicle. Using the reported capital costs from Borlaug et al. and the number of vehicles per charger, we estimate the average capital cost as: 

\begin{equation*}
C_{overall}=[\frac{C_{L2res}}{VMT \cdot FE \cdot VPC_{res} }]+[\frac{C_{L2public}}{VMT \cdot FE \cdot VPC_{L2public}}]+[\frac{C_{DCFC}}{VMT \cdot FE \cdot VPC_{DCFC}}]
\end{equation*}

Where VMT is the annual vehicle miles traveled by a light-duty vehicle (assumed to be 11,325 based on EPA MARKAL), FE is the average fuel economy (0.76 billion vehicle miles traveled per PJ), and VPC is the number of vehicles served per charger and Ci are the capital costs for a given charger type. Using base-year VPC assumptions, we obtain a 2020 overall capital cost of 317 \\$M/(PJ/year). O&M costs are assumed to be 1\% of total capital costs, in accordance with Borlaug et al (2020). For years beyond 2020, we assume a higher VPC. The 2021 NREL report estimates that by 2030, each public L2 charger will serve 25 vehicles and each DCFC charger will serve 550 vehicles. We assume a slight increase in vehicles served by L2 home chargers, increasing the VPC from 1 to 1.3. This increased utilization yields a capital cost in 2030 of 205.13 $M/(PJ/year). We assume a linear decrease from 2020 to 2030 and make the assumption that capital costs stay flat from 2030 onward. Under deep decarbonization, it is likely that capital costs will fall and utilization rates will increase, so our values likely represent a conservative estimate. 

Far less data exists about heavy-duty charger usage rates and capital costs. Borlaug et al. (2021) estimates the fraction of heavy-duty EVs operating in fleets that could charge using existing electric distribution systems and the fraction needing infrastructure updates, such as transformers, additional substations, and upgrades to feeder circuits. The study also reports estimates for the costs of necessary transmission infrastructure upgrades as well as 50, 150, and 350 kW DCFCs. The authors find that in the fleets studied, the majority of electric substations can supply 100 battery electric trucks with 100 kW per vehicle without additional upgrades. The study also finds that most heavy-duty stations could charge their trucks using typical light-duty chargers (50 kW DCFC) but provides cost estimates for 150 kW and 350 kW DCFCs. While data does not exist on the fraction of fleet owners that will adopt each charger type, we assume that most fleet owners will opt for the least expensive charger type. Specifically, we assume that 75% of substations will be equipped with 50 kW DCFCs, 20% will use 150 kW DCFCs, and the remaining 5% will use 350 kW DCFCs. The paper also reports that approximately 6% of stations need new transformers, 3% need upgraded feeder circuits, 6% need new substations, and 3% need new feeder breakers; we include these estimated retrofit costs in our overall capital estimate. The vehicles in the fleets studied in Borlaug et al.’s analysis were parked for an average of 14 hours per day. Assuming an average energy consumption of 2 kWh mile-1, it would take ~6 hours to charge a truck with a range of 300 miles. We assume, therefore, that each charger could serve 2 trucks during the 14 parked hours. Lastly, Borlaug et al. report that medium-duty and short-haul heavy duty fleets in the US are rarely comprised of more than 100 vehicles, so we assume that each station has 50 plugs (capable of charging 100 vehicles). The total cost (in million USD) of a single station is calculated as: 

\begin{equation*}
C_{total} = [(FP_{50kW} \cdot C_{50kW}+FP_{150kW} \cdot C_{150kW}+FP_{350kW} \cdot C_{350kW})×PPS]+[FS_{transformer} \cdot C_{transformer}+FS_{feeder circuit} \cdot C_{feeder circuit}+FS_{substation} \cdot C_{substation}+FS_{feeder breaker} \cdot C_{feeder breaker}]
\end{equation*}

Where FPi is the fraction of plugs at each charging level, PPS is the number of plugs per station,  FSi is the fraction of stations needing each kind of upgrade, and Ci is the cost of each component. We find the average capital cost for a new charging depot to be $8.9 million. Assuming 27,390 vehicle miles traveled per year per vehicle (based on EPA MARKAL) and an average energy consumption of 0.11 PJ per billion vmt, we calculate a capital cost of 17,160 \\$M/(PJ/year). As with light-duty EVSE, we assume the O&M costs are 1% of the capital cost.


##### Hydrogen Refueling Stations

We draw most of our assumptions for hydrogen refueling capital costs from Argonne National Laboratory’s Light- and Heavy-Duty Hydrogen Refueling Station Analysis Models (HRSAM and HDRSAM). The models provide capital and operation & maintenance (O&M) costs for stations with user-specified capacities. While a number of the input parameters (such as storage pressure and temperature and number of compressor stages) may be modified by the user, we use the default values provided in the model spreadsheets. The only parameter we modify is the station capacity. Capacity assumptions and the resulting station costs are detailed below. 


To estimate the capital cost of light-duty refueling infrastructure, HRSAM requires a station capacity input [kg H2/day], as station cost does not scale linearly with hydrogen capacity. A 2020 DOE program record reports that the average station capacity for stations operating between 2012 and 2017 was 480 kg/day, but newer planned gaseous H2 stations have average capacities of 770 kg/day. We assume hydrogen refueling stations constructed in 2020 will have a capacity of 480 kg/day, rising to 770 kg/day in 2030, and continuing to increase linearly to 1,350 kg/day in 2050. To calculate capital cost [\\$M/(PJ/year)], we divide the cost output from HRSAM [\\$M] by the annual station capacity [PJ/year]. Capital costs range from 210.5 \\$M/(PJ/year) in 2020 to 95.4 \\$M/(PJ/year) in 2050. Fixed operation & maintenance costs also come from HRSAM and range from 13.1 \\$M/PJ in 2020 to 6.7 $M/PJ in 2050.


As hydrogen heavy-duty trucks are largely still in the development phase, limited information exists on possible refueling station capacities. One 2019 study evaluating the costs of US hydrogen refueling infrastructure models three capacity levels: 756 kg/day, 3024 kg/day, and 7560 kg/day. Their lower bound estimate is similar to existing capacities for light-duty refueling stations, while the upper bound lines up well with planned future commercial deployment. According to press releases, the FCEV manufacturer, Nikola, plans to develop stations with an 8 ton/day (7257 kg/day) capacity. As such, we assume stations built in 2020 have a 770 kg/day capacity (identical to currently planned light-duty vehicle refueling stations) rising linearly to 7600 kg/day in 2050. Cost estimates are drawn from Argonne National Laboratory’s Heavy-Duty Refueling Station Analysis Model (HDRSAM). Capital costs range from 408.1 \\$M/(PJ/year) in 2020 to 104.4 \\$M/(PJ/year) in 2050. Fixed costs range from 25.1 \\$M/PJ in 2020 to 10.3 \\$M/PJ in 2050.



#### 1.6 Technology/commodity description lookup tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. LDV, shipping, rail). The tool provides a list of all the technologies in the database that may be relevant to the query.

In [14]:
w = widgets.Text(value='LDV')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='LDV')

Output()

#### 1.7 Network diagram lookup tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [15]:
w = widgets.Text(value='TMDHDV_WTP')
display(w)
def f(w):
    if len(w)>0:
        df = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df['comm_desc'] = df['comm_desc'].str.replace('#','').str.strip()
    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['comm_desc'][0]))
                final_dem = df['comm_name'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="12,12";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=0,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)


Text(value='TMDHDV_WTP')

Output()

#### 1.8 Technology/commodity look-up tool <a class="anchor" id="lookup"></a>
Use the tool below to retrieve the description for any technology or commodity within the database. Type the commodity or technology name in the box below to view the description. Note that names are case sensitive.

In [16]:
w = widgets.Text(value='ELC')
display(w)
def f(w):
    df = pd.read_sql("SELECT * FROM commodities WHERE comm_name='" + w + "'", con)
    if len(df)==0:
        df = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)

    if len(df)>0:
        try:
            display(Markdown((df['comm_desc'].values[0].replace('#', '').strip())))
        except:
            display(Markdown(df['tech_desc'].values[0].replace('#', '').strip()))
    else:
        print('')


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='ELC')

Output()

#### 1.9 Demand Technology Specification <a class="anchor" id="demand_technologies"></a>

Techno-economic parameters for an individual technology are assumed to be the same for all regions. Regional demand and existing capacity allocation is described in the preceeding sections. Relevant techno-economic paramters can be queried and displayed in the sections that follow. 

##### 1.9.1 Investment costs <a class="anchor" id="cost_invest"></a>

In [17]:
def show_cost_invest(con):
    
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)

    df = pd.read_sql(
        "SELECT regions, tech,vintage, cost_invest, cost_invest_units FROM CostInvest WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport' ) ORDER BY tech, vintage",
        con)

    display_types = ['table', 'figure']
    
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()


    def filter_tech(tech='', region='', o_format=''):
        if tech == 'All':
            df_sel = df[(df.regions == region)]
        else:
            df_sel = df[(df.agg_tech.str.contains(tech)) & (df.regions == region)]
        if o_format == 'table':
            df_sel = df_sel.pivot_table(
                index=['regions', 'tech', 'agg_tech', 'cost_invest_units'],
                columns='vintage',
                values='cost_invest').reset_index().set_index('regions')
            df_sel.rename(columns={'cost_invest_units': 'units', 'agg_tech':'description'}, inplace=True)
            df_sel['units'] = df_sel['units'].str.replace('#','').str.replace('M$','$M').str.strip()
            if len(df_sel['units'].unique())==1:
                df_sel['units'] = df_sel['units'].unique()[0].replace('M$','$M')
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif (o_format == 'figure') & (tech!='All') :
            fig, ax = plt.subplots(figsize=(10, 6))
            for ind_tech in df_sel.tech.unique():
                plt.plot(df_sel[df_sel.tech == ind_tech].vintage,
                         df_sel[df_sel.tech == ind_tech].cost_invest,
                         label=ind_tech)
            plt.legend()
            plt.ylabel('Investment costs (' + df_sel['cost_invest_units'].unique()[0].replace('#','').strip().replace('M$','$M') + ')')
            #plt.ylim([0, df.cost_invest.max() * 1.1])
            plt.xlabel('Vintage')


    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_invest(con)

##### 1.9.2 Fixed costs <a class="anchor" id="cost_fixed"></a>

In [18]:
def show_cost_fixed(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech,vintage, cost_fixed_units, AVG(cost_fixed) AS cost_fixed FROM CostFixed WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport') GROUP BY regions, tech,vintage ORDER BY tech, vintage", con)
    display_types = ['table', 'figure']
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())

    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))    
    regions = [x for x in df.regions.unique() if '-' not in x]
    def filter_tech(tech ='', region = '', o_format=''):
        if tech=='All':
            df_sel = df[(df.regions==region)].copy()
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        if o_format=='table':
            #df_sel = df_sel.pivot_table(index=['regions','tech', 'cost_fixed_units'], columns='vintage', values='cost_fixed').reset_index().set_index('regions')
            df_sel = df_sel[['regions','tech','agg_tech','vintage','cost_fixed_units','cost_fixed']]
            df_sel['cost_fixed_units'] = df_sel['cost_fixed_units'].str.replace('#','').str.replace('M$','$M').str.strip()
            header = ['regions', 'technology', 'description','vintage', 'units', 'fixed cost']
            display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header,  floatfmt=".0f" , tablefmt='html')))

        elif o_format=='figure':
            fig, ax = plt.subplots(figsize=(10,6))
            for ind_tech in df_sel.tech.unique():
                plt.bar(df_sel[df_sel.tech==ind_tech].vintage, df_sel[df_sel.tech==ind_tech].cost_fixed, label=ind_tech)
            plt.legend()
            if len(df_sel)==1:
                plt.xlim([df_sel.vintage.values - 2, df_sel.vintage.values + 2])
                plt.xticks(np.arange(df_sel.vintage.values - 1, df_sel.vintage.values + 2))
            plt.ylabel('Fixed costs ($M/GWyr)')
            #plt.ylim([0, df.cost_fixed.max()*1.1])
            plt.xlabel('Vintage')

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_fixed(con)

##### 1.9.3 Variable costs <a class="anchor" id="cost_variable"></a>

In [19]:
def show_cost_variable(con):

    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)

    df = pd.read_sql("SELECT regions, tech,vintage, cost_variable_units, AVG(cost_variable) AS cost_variable FROM CostVariable WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport') AND cost_variable> 0 GROUP BY regions, tech,vintage ORDER BY vintage", con)
    display_types = ['table', 'figure']
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())

    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech ='', region = '', o_format=''):
        if tech=='All':
            df_sel = df[(df.regions==region)].copy()
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        if o_format=='table':
            #df_sel = df_sel.pivot_table(index=['regions','tech', 'cost_variable_units'], columns='vintage', values='cost_variable').reset_index().set_index('regions')
            df_sel = df_sel[['regions','tech','agg_tech','vintage','cost_variable_units','cost_variable']]
            header = ['regions', 'technology', 'decription','vintage', 'units', 'variable cost']
            display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header,  floatfmt=".2f" , tablefmt='html')))
        elif o_format=='figure':
            fig, ax = plt.subplots(figsize=(10,6))
            for ind_tech in df_sel.tech.unique():
                plt.bar(df_sel[df_sel.tech==ind_tech].vintage, df_sel[df_sel.tech==ind_tech].cost_variable, label=ind_tech)
            plt.legend()
            if len(df_sel)==1:
                plt.xlim([df_sel.vintage.values - 2, df_sel.vintage.values + 2])
                plt.xticks(np.arange(df_sel.vintage.values - 1, df_sel.vintage.values + 2))
            plt.ylabel('Variable costs ($M/PJ)')
            #plt.ylim([0, df.cost_variable.max()*1.1])
            plt.xlabel('Vintage')
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)

show_cost_variable(con)

##### 1.9.4 Efficiency <a class="anchor" id="efficiency"></a>

In [20]:
def show_efficiency(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, efficiency FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df#df.drop("vintage", axis=1).groupby(by = ['regions','agg_tech']).sum().reset_index()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
            df_sel.efficiency = 100*df_sel.efficiency
        df_sel = df_sel[['regions','tech','agg_tech','vintage','efficiency']]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description","vintage",'efficiency (%)' ], floatfmt=".1f", tablefmt='html')))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_efficiency(con)

##### 1.9.5 Existing Capacity <a class="anchor" id="exist_cap"></a>

In [21]:
def show_exist_cap(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech,vintage, exist_cap, exist_cap_units FROM ExistingCapacity WHERE tech IN (SELECT tech FROM technologies WHERE sector=='transport')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df.drop("vintage", axis=1).groupby(by = ['regions','tech','agg_tech','exist_cap_units']).sum().reset_index()
    df_sum.sort_values(by='exist_cap', ascending=False, inplace=True)
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description", "units","capacity" ], floatfmt=".1f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_exist_cap(con)


##### 1.9.6 Discount Rate <a class="anchor" id="discount_rate"></a>

In [22]:
def show_disc_rate(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, tech_rate FROM DiscountRate \
    WHERE tech in (SELECT tech FROM technologies WHERE sector='transport')", con)
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        if tech=='All':
            df_sel = df[(df.regions==region)]
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        df_sel = df_sel.pivot_table(index=['regions','tech', 'agg_tech'], columns='vintage', values='tech_rate').reset_index()
        df_sel.rename(columns={'agg_tech':'description'},inplace=True)
        header = list(df_sel.columns.values)
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header, floatfmt=".3f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_disc_rate(con)